In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lsst.rsp import get_tap_service
sys.path.append(os.path.abspath('../../../sso_query/'))
import query as q
import query_helpers as qh

In [2]:
service = get_tap_service("ssotap")
assert service is not None
results = service.search("SELECT column_name, datatype, description, "
                         "unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp03_catalogs_10yr.DiaSource'")
results.to_table().to_pandas()

,column_name,datatype,description,unit
0,band,char,Name of the band used to take the exposure whe...,
1,ccdVisitId,int,Id of the ccdVisit where this diaSource was me...,
2,dec,double,Dec-coordinate of the center of this diaSource.,deg
3,decErr,float,Uncertainty of dec.,deg
4,decTrue,double,True (noiseless) declination of the simulated ...,
5,diaObjectId,long,Id of the diaObject this source was associated...,
6,diaSourceId,long,Unique id.,
7,mag,float,Magnitude. This is a placeholder and should be...,
8,magErr,float,Magnitude error. This is a placeholder and sho...,
9,magTrueVband,float,True (noiseless) V-band magnitude of the simul...,


In [ ]:
results['description'][9]

### Query testing

Object type given, no join

Object type given, join

Object type not given, no join

Object type not given, join

#### Object type given, no join

In [11]:
query, object_type = q.make_query_general(object_type = "JFC")
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "NEO")
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "TNO")
print(query)
print(object_type)

SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID FROM dp03_catalogs_10yr.MPCORB as mpc
    WHERE (mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e)) >= 0 AND (5.204 * (1 - mpc.e)) / mpc.q + 2 * COS(RADIANS(mpc.incl)) * SQRT((mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e))) BETWEEN 2.0 AND 3.0;
JFC
SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID FROM dp03_catalogs_10yr.MPCORB as mpc
    WHERE mpc.q < 1.3 AND mpc.e < 1.0 AND mpc.q/(1-mpc.e) < 4.0;
NEO
SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID FROM dp03_catalogs_10yr.MPCORB as mpc
    WHERE mpc.q/(1-mpc.e) > 30.1 AND mpc.q/(1-mpc.e) < 50;
TNO


#### Object type given, join

In [9]:
query, object_type = q.make_query_general(object_type = "JFC", join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "NEO", join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "Ntrojan", join = 'Diasource')
print(query)
print(object_type)

SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID, dias.magTrueVband, dias.band FROM dp03_catalogs_10yr.MPCORB as mpc
    INNER JOIN dp03_catalogs_10yr.DiaSource AS dias ON mpc.ssObjectId = dias.ssObjectId
    WHERE (mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e)) >= 0 AND (5.204 * (1 - mpc.e)) / mpc.q + 2 * COS(RADIANS(mpc.incl)) * SQRT((mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e))) BETWEEN 2.0 AND 3.0;
JFC
SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID, dias.magTrueVband, dias.band FROM dp03_catalogs_10yr.MPCORB as mpc
    INNER JOIN dp03_catalogs_10yr.DiaSource AS dias ON mpc.ssObjectId = dias.ssObjectId
    WHERE mpc.q < 1.3 AND mpc.e < 1.0 AND mpc.q/(1-mpc.e) < 4.0;
NEO
SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID, dias.magTrueVband, dias.band FROM dp03_catalogs_10yr.MPCORB as mpc
    INNER JOIN dp03_catalogs_10yr.DiaSource AS dias ON mpc.ssObjectId = dias.ssObjectId
    WHERE mpc.q/(1-mpc.e) > 29.8 AND mpc.q/(1-mpc.e) < 30.4;
Ntrojan


#### Params given, join

In [4]:
query, object_type = q.make_query_general(t_cutoff_min = 2.0, t_cutoff = 3.0, join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(a_cutoff_min = 30.1, a_cutoff = 50)
print(query)
print(object_type)

SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID, dias.magTrueVband, dias.band FROM dp03_catalogs_10yr.MPCORB as mpc
    INNER JOIN dp03_catalogs_10yr.DiaSource AS dias ON mpc.ssObjectId = dias.ssObjectId
    WHERE (mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e)) >= 0 AND (5.204 * (1 - mpc.e)) / mpc.q + 2 * COS(RADIANS(mpc.incl)) * SQRT((mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e))) BETWEEN 2.0 AND 3.0;
JFC
SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID FROM dp03_catalogs_10yr.MPCORB as mpc
    WHERE mpc.q/(1-mpc.e) > 30.1 AND mpc.q/(1-mpc.e) < 50;
TNO


In [3]:
query, object_type = q.make_query_general(object_type = "JFC", join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(t_cutoff_min = 2.0, t_cutoff = 3.0, join = 'Diasource')
print(query)
print(object_type) 

SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID, dias.magTrueVband, dias.band FROM dp03_catalogs_10yr.MPCORB as mpc
    INNER JOIN dp03_catalogs_10yr.DiaSource AS dias ON mpc.ssObjectId = dias.ssObjectId
    WHERE (mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e)) >= 0 AND (5.204 * (1 - mpc.e)) / mpc.q + 2 * COS(RADIANS(mpc.incl)) * SQRT((mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e))) BETWEEN 2.0 AND 3.0;
JFC
SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID, dias.magTrueVband, dias.band FROM dp03_catalogs_10yr.MPCORB as mpc
    INNER JOIN dp03_catalogs_10yr.DiaSource AS dias ON mpc.ssObjectId = dias.ssObjectId
    WHERE (mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e)) >= 0 AND (5.204 * (1 - mpc.e)) / mpc.q + 2 * COS(RADIANS(mpc.incl)) * SQRT((mpc.q * (1 - mpc.e)) / (5.204 * (1 + mpc.e))) BETWEEN 2.0 AND 3.0;
JFC


#### Testing join with DP1 DiaSource

In [2]:
string, class_name = q.make_query(catalog = "dp1", class_name = "NEO", cutoffs = None, join = "DiaSource")
print(string)
# NEO_data_table = q.run_query(string, class_name, "dp1", to_pandas = True)

# mags = q.calc_magnitude(NEO_data_table['apFlux'])
# NEO_data_table['Magnitude'] = mags

Querying dp1.DiaSource for: ['dias.apFlux', 'dias.apFlux_flag', 'dias.apFluxErr', 'dias.band']
SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID, mpc.mpcDesignation, dias.apFlux, dias.apFlux_flag, dias.apFluxErr, dias.band FROM dp1.MPCORB AS mpc
    INNER JOIN dp1.DiaSource AS dias ON mpc.ssObjectId = dias.ssObjectId
    WHERE mpc.q < 1.3 AND mpc.e < 1.0 AND mpc.q/(1-mpc.e) < 4.0;


In [5]:
NEO_data_table.columns

Index(['apFlux', 'apFluxErr', 'apFlux_flag', 'band', 'e', 'incl',
       'mpcDesignation', 'q', 'ssObjectID', 'a', 'class_name', 'Magnitude'],
      dtype='object')

In [4]:
service = get_tap_service("tap")
assert service is not None
results = service.search("SELECT column_name, datatype, description, "
                         "unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp1.DiaSource'")
results.to_table().to_pandas()

,column_name,datatype,description,unit
0,apFlux,float,Flux in a 12 pixel radius aperture on the diff...,nJy
1,apFlux_flag,boolean,General aperture flux algorithm failure flag; ...,
2,apFlux_flag_apertureTruncated,boolean,Aperture did not fit within measurement image.,
3,apFluxErr,float,Estimated uncertainty of apFlux.,nJy
4,band,char,Band used to take this observation.,
...,...,...,...,...
82,visit,long,Id of the visit where this diaSource was measu...,
83,x,double,"Unweighted first moment centroid, overall cent...",pixel
84,xErr,float,1-sigma uncertainty on x position,pixel
85,y,double,"Unweighted first moment centroid, overall cent...",pixel


#### Testing join with DP1 SSObject

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lsst.rsp import get_tap_service
sys.path.append(os.path.abspath('../../../sso_query/'))
import query as q
import query_helpers as qh

string, class_name = q.make_query(catalog = "dp03_catalogs_10yr", class_name = "NEO", cutoffs = None, join = "SSObject")
print(string)
# NEO_data_table = q.run_query(string, class_name, "dp1", to_pandas = True)


Querying dp03_catalogs_10yr.SSObject for: ['sso.g_H', 'sso.r_H', 'sso.i_H', 'sso.discoverySubmissionDate', 'sso.numObs']
SELECT mpc.incl, mpc.q, mpc.e, mpc.ssObjectID, mpc.mpcDesignation, sso.g_H, sso.r_H, sso.i_H, sso.discoverySubmissionDate, sso.numObs, (sso.g_H - sso.r_H) AS g_r_color, (sso.r_H - sso.i_H) AS r_i_color FROM dp03_catalogs_10yr.MPCORB AS mpc
    INNER JOIN dp03_catalogs_10yr.SSObject AS sso ON mpc.ssObjectId = sso.ssObjectId
    WHERE mpc.q < 1.3 AND mpc.e < 1.0 AND mpc.q/(1-mpc.e) < 4.0;
